In [1]:
# Importando Librerías
import re
import json
import time
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)


In [ ]:
# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

In [2]:
# Creando WebDriver para la busqueda con Chrome
driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
#Ingresando a página para obtener información de las partes de Jojo's
page_url = f'https://jojowiki.com/Category:JoJo%27s_Bizarre_Adventure_Parts'
driver.get(page_url)

## Hallando URL/Imagen cada parte 

In [4]:
# Encontrando partes
jojo_images = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[1]/a/img')
jojo_url = driver.find_elements(
    By.XPATH, '//*[@id="mw-content-text"]/div[2]/div[1]/div/div[2]/div/div[2]/a'
)

jojo_data = {}
for image, url in zip(jojo_images, jojo_url):

    title = url.text
    url = url.get_attribute('href')
    image = image.get_attribute("src")
    jojo_data[title] = {'url': url}
    jojo_data[title]['image'] = image

# with open('data/parts/jojo_parts.json', 'w', encoding='utf-8') as file:
#     json.dump(jojo_data, file, indent=4, sort_keys=False)

jojo_data

{'Phantom Blood': {'url': 'https://jojowiki.com/Phantom_Blood',
  'image': 'https://static.jojowiki.com/images/thumb/b/b1/latest/20191015213555/Volume_1.jpg/200px-Volume_1.jpg'},
 'Battle Tendency': {'url': 'https://jojowiki.com/Battle_Tendency',
  'image': 'https://static.jojowiki.com/images/thumb/3/34/latest/20191015213237/Volume_11.jpg/200px-Volume_11.jpg'},
 'Stardust Crusaders': {'url': 'https://jojowiki.com/Stardust_Crusaders',
  'image': 'https://static.jojowiki.com/images/thumb/0/0a/latest/20191015212952/Volume_17.jpg/200px-Volume_17.jpg'},
 'Diamond is Unbreakable': {'url': 'https://jojowiki.com/Diamond_is_Unbreakable',
  'image': 'https://static.jojowiki.com/images/thumb/b/ba/latest/20191015213904/Volume_36.jpg/200px-Volume_36.jpg'},
 'Vento Aureo': {'url': 'https://jojowiki.com/Vento_Aureo',
  'image': 'https://static.jojowiki.com/images/thumb/4/40/latest/20191015215618/Volume_49.jpg/200px-Volume_49.jpg'},
 'Stone Ocean': {'url': 'https://jojowiki.com/Stone_Ocean',
  'image'

## Hallando Info por Parte

In [5]:
for part in jojo_data.keys():
    driver.get(jojo_data[part]['url'])
    fields = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[2]/aside/div/h3')
    datum = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/aside/div/div')
    for field, data in zip(fields, datum):
        resume = ' '.join([driver.find_element(By.XPATH, f'//*[@id="mw-content-text"]/div[2]/p[{i}]').text for i in range(3,6)])
        resume = re.sub("\[.*?\]", "", resume)
        jojo_data[part]['resume'] = resume
        jojo_data[part][field.text.lower()] = data.text


In [6]:
jojo_data

{'Phantom Blood': {'url': 'https://jojowiki.com/Phantom_Blood',
  'image': 'https://static.jojowiki.com/images/thumb/b/b1/latest/20191015213555/Volume_1.jpg/200px-Volume_1.jpg',
  'resume': "Phantom Blood (ファントムブラッド Fantomu Buraddo) is the first part of JoJo's Bizarre Adventure. It was originally serialized in Weekly Shonen Jump from December 1986 to October 1987 under the title of JoJo's Bizarre Adventure Part 1 Jonathan Joestar: His Youth, and was collected into five volumes by Jump Comics. The title Phantom Blood was first officially used in the book JOJO A-GO!GO!, and then later used for the Bunkoban release of the manga. The part has received two anime adaptations: a 2007 film by Studio A.P.P.P. and a 2012 TV series by David Production. It was also adapted into a 2006 PlayStation 2 game by Anchor Inc. and Bandai. The story follows Jonathan Joestar as he matures with and eventually battles his adoptive brother, the cunning and merciless Dio Brando.",
  'japanese name': 'ジョジョの奇妙な冒険\